In [28]:
import joblib

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
import re

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet

In [29]:
df = pd.read_csv("../data/hate_classes.tsv", sep='\t')
# process text for embeddings
text = df.Text.apply(basic_tokenize)

tokenizer = Tokenizer(num_words=20000, split=' ', oov_token='<unw>', filters='')
tokenizer.fit_on_texts(text.values)
X = tokenizer.texts_to_sequences(text.values)
X = pad_sequences(X, maxlen=200)

Y = []
for row in df.itertuples():
    Y.append((row.RAE,row.NAT,row.SXO,row.GEN,row.REL,row.IDL,row.POL,row.MPH))
Y = np.array(Y)

In [47]:
model = keras.models.load_model('../models/hate_classification.h5')
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)

result = model.evaluate(x_test, y_test)

486/486 [==============================] - 0s 339us/sample - loss: 1.1395 - binary_accuracy: 0.9689


In [46]:
df = pd.read_csv("../data/hate_sanity_check.tsv", sep='\t')
text = df.tweet.apply(basic_tokenize)
tokenizer = Tokenizer(num_words=20000, split=' ', oov_token='<unw>', filters='')
tokenizer.fit_on_texts(text.values)
T = tokenizer.texts_to_sequences(text.values)
T = pad_sequences(T, maxlen=200)

sample = 67
result = model.predict(T)
result = np.round(result)

print(result[sample])
# race, nationality, sex orentation, gender, religion, political identity, mental status
# (row.RAE,row.NAT,row.SXO,row.GEN,row.REL,row.IDL,row.POL,row.MPH)
print(df.loc[sample,:].tweet)

[1. 0. 0. 0. 0. 0. 0. 0.]
&#8220;@Maxwell_Larsen: When you live this gay, you can't be afraid of two black dicks in your butt" you fucking queer
